<img src="EPITA.png" alt="Epita" width="200"/>

## Arithmétique pour la cryptographie

Cours n ${ }^{\circ} 5$
EPITA Cyber 1 2024-2025

Dans ce cours, nous présenterons le modèle mathématique sous-jacent au protocole de chiffrement asymétrique RSA (Rivest–Shamir–Adleman). Ce cinquième cours présente le protocole RSA (génération de clé privée et publique, fonction de chiffrement, justification mathématique).

## 1 Rappel : Schéma d'un Chiffrement Asymétrique

Le schéma de fonctionnement d'un protocole asymétrique repose sur cinq étapes. Dans l'exemple suivant, l'agent Bob souhaite envoyer des données à l'agent Alice.
![](https://cdn.mathpix.com/cropped/2025_05_06_6988509eb63a7c3260a9g-1.jpg?height=538&width=1700&top_left_y=1216&top_left_x=189)

1. Alice génère une **clé privée** et une **clé publique**.
2. Alice transmet la **clé publique** à Bob, via un canal de communication quelconque.
3. Bob chiffre son message $m$ avec la clé publique d’Alice.
4. Bob transmet le message chiffré $c$ via un canal de communication non sécurisé.
5. Alice déchiffre le message $c$ avec sa **clé privée**, et récupère $m$.

Figure 1 - Etapes d'un Protocole Asymétrique : Bob transmet des informations à Alice.

## 2 Le protocole RSA

### 2.1 Introduction

Le protocole RSA est un exemple célèbre de chiffrement asymétrique. Ce protocole utilise de grands nombres comme clé privée et clé publique. Ces clés sont qénérées de manière à garantir les propriétés d'une bonne fonction de chiffrement.

### 2.2 Génération des clés

La génération des clés se décompose en 5 étapes :

1. Générer aléatoirement deux nombres premiers distincts, noté $p$ et $q$.
2. Calculer le module de chiffrement, noté $n$, définie par $n=p * q$.
3. Calculer l'indicatrice d'Euler de $n$, noté $\phi(n)$, et définie par $\phi(n)=(p-1)(q-1)$. Cette valeur désigne le nombre d'entiers inférieur à $n$ et premier avec $n$.
4. Choisir l'exposant de chiffrement, noté $e$, tel que $e$ est un entier naturel, inférieur à $\phi(n)$ et premier avec $\phi(n)$.
5. Calculer l'exposant de déchiffrement, noté $d$, définie comme l'inverse modulaire de $e$ pour la multiplication modulo $\phi(n)$. L'entier $d$ doit être positif et inférieur à $\phi(n)$.

La paire ( $n, e$ ) constitue la clé de chiffrement, et la paire ( $n, d$ ) constitue la clé de déchiffrement (voir section suivante Fonction de chiffrement et déchiffrement).

### 🔐 Exemple de chiffrement RSA

#### 🧾 Étape 1 : Génération des clés (par Alice)

1. Alice choisit deux nombres premiers :
   $p = 17$, $q = 11$

2. Elle calcule :

   $$
   n = p \cdot q = 17 \cdot 11 = 187
   $$

3. Elle calcule :

   $$
   \varphi(n) = (p - 1)(q - 1) = 16 \cdot 10 = 160
   $$

4. Elle choisit $e = 7$ tel que $pgcd(e, \varphi(n)) = 1$

5. Elle calcule l’inverse modulaire de $e$ modulo $\varphi(n)$ :

   $$
   d = e^{-1} \mod 160 = 23
   $$

   (car $7 \cdot 23 \mod 160 = 1$)

🔓 **Clé publique** : $(n = 187, e = 7)$
🔐 **Clé privée** : $(n = 187, d = 23)$

In [ ]:
# Génération des clés RSA (Alice)

from math import gcd

def modinv(a, m):
    '''Calcul de l'inverse modulaire en utilisant l'algorithme d'Euclide étendu.'''
    m0, x0, x1 = m, 0, 1
    if gcd(a, m) != 1:
        raise ValueError("L'inverse n'existe pas")
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

# Étape 1 : Choisir les nombres premiers p et q
p = 13
q = 7
print(f"Étape 1 : Nombres premiers choisis p={p}, q={q}")

# Étape 2 : Calculer n
n = p * q
print(f"Étape 2 : Calculer n = p * q = {n}")

# Étape 3 : Calculer φ(n)
phi_n = (p - 1) * (q - 1)
print(f"Étape 3 : Calculer φ(n) = (p-1)*(q-1) = {phi_n}")

# Étape 4 : Choisir e tel que pgcd(e, φ(n)) = 1
e = 5
if gcd(e, phi_n) != 1:
    raise ValueError("e doit être premier avec φ(n)")
print(f"Étape 4 : Choix de e = {e}")

# Étape 5 : Calculer d (inverse modulaire de e modulo φ(n))
d = modinv(e, phi_n)
print(f"Étape 5 : Calculer d = e^(-1) mod φ(n) = {d}")

# Clés publique et privée
public_key = (n, e)
private_key = (n, d)
print(f"Clé publique : {public_key}")
print(f"Clé privée : {private_key}")

Étape 1 : Nombres premiers choisis p=17, q=11
Étape 2 : Calculer n = p * q = 187
Étape 3 : Calculer φ(n) = (p-1)*(q-1) = 160
Étape 4 : Choix de e = 7
Étape 5 : Calculer d = e^(-1) mod φ(n) = 23
Clé publique : (187, 7)
Clé privée : (187, 23)


## Remarques :

- La clé publique ( $n, e$ ) contient le module de chiffrement $n$, qui est donc une information publique. La décomposition de $n$ en $p * q$ doit rester secrète et est difficile à calculer pour de grands nombres.
- Les entiers $p$ et $q$ n'interviennent pas directement dans les fonctions de chiffrement et déchiffrement.


### 2.3 Fonction de chiffrement et déchiffrement

Les étapes de chiffrement et de déchiffrement s'effectuent à l'aide des clés publiques et privées par exponentiation modulaire du message.
Nous considérons que le message à chiffrer $m$ est un entier.
Etant donné le module de chiffrement $n$, l'exposant de chiffrement $e$, l'exposant de déchiffrement $d$ et un message représenté par un entier $m$.
Le chiffrement du message $m$ s'effectue par l'exponentation modulaire de $m$ avec l'exposant $e$ et modulo $n$ :

$$
m^{e} \equiv c \quad(\bmod n) .
$$

Le déchiffrement du message $c$ s'effectue par l'exponentation modulaire de $c$ avec l'exposant $d$ et modulo $n$ :

$$
c^{d} \equiv m \quad(\bmod n) .
$$

Remarques : Validité des paramètres $n, e, d, m$.

- Le chiffrement et le déchiffrement s'effectue modulo $n$, l'entier à chiffrer $m$ doit donc être tel que $0 \leq m<n$.
- L'entier à chiffrer et les exposants doivent être assez grands pour que l'exponentiation du message soit supérieur à $n$ et que l'opérateur modulo s'applique.


## Exemple :

1. Soit :
   - Le module de chiffrement **\(n = 33\)**
   - L'exposant de chiffrement **\(e = 3\)**
   - L'exposant de déchiffrement **\(d = 7\)**
   - Le message **\(m = 20\)**
 
Le message chiffré est $c \equiv m^{e} \equiv 20^{3} \equiv 8000 \equiv 14(\bmod 33)$.

In [ ]:
# Chiffrement du message
m = 20
n = 33
e = 3
c = (m ** e) % n
print(f"Message chiffré c = m^e mod(n) = {m}^{e} mod {n} = {c}")

Message chiffré c = m^e mod(n) = 20^3 mod 33 = 14


Avec le message chiffré $c$, le message original est $\left(c^{d}\right) \equiv 14^{7} \equiv 105413504 \equiv 20(\bmod 33)$.

In [ ]:
# Déchiffrement du message
m_retrouve = (c ** d) % n
print(f"Message déchiffré m = c^d mod n = {c}^{d} mod {n} = {m_retrouve}")

Message déchiffré m = c^d mod n = 14^29 mod 33 = 26


2. Soit le module de chiffrement **($n=91$)**,
   - l'exposant de chiffrement **$(e=5$)**,
   - l'exposant de déchiffrement **($d=29$)**,
   - et le message **($m=10$)**.

In [ ]:
# Chiffrement du message
n = 91
e = 5
m = 10
d = 29
c = (m ** e) % n
print(f"Message chiffré c = m^e mod(n) = {m}^{e} mod {n} = {c}")

Message chiffré c = m^e mod(n) = 10^5 mod 91 = 82


Le message chiffré est $c \equiv m^{e} \equiv 10^{5} \equiv 82(\bmod 91)$.
- A partir du message chiffré $c$, le message original est $\left(c^{d}\right) \equiv 82^{29} \equiv 10(\bmod 91)$.

In [ ]:
# Déchiffrement du message
m_retrouve = (c ** d) % n
print(f"Message déchiffré m = c^d mod n = {c}^{d} mod {n} = {m_retrouve}")

Message déchiffré m = c^d mod n = 82^29 mod 91 = 10


## Exercices :

1. Soit le module de chiffrement $n=33$,
   - l'exposant de chiffrement $e=3$,
   - et le message $m=5$.
   - Calculer le message chiffré.

In [ ]:
# Chiffrement du message
n = 33
e = 3
m = 5
c = (m ** e) % n
print(f"Message chiffré c = m^e mod(n) = {m}^{e} mod {n} = {c}")

Message chiffré c = m^e mod(n) = 5^3 mod 33 = 26


2. Soit le module de chiffrement $n=35$,
   - l'exposant de déchiffrement $d=5$,
   - et le message chiffré $c=3$. Déterminer le message original.

In [ ]:
# Chiffrement du message
n = 35
d = 5
c = 3
print(f"Message chiffré c = m^e mod(n) = {m}^{e} mod {n} = {c}")

In [ ]:
# Déchiffrement du message
m_retrouve = (c ** d) % n
print(f"Message déchiffré m = c^d mod n = {c}^{d} mod {n} = {m_retrouve}")

Message déchiffré m = c^d mod n = 26^29 mod 33 = 14


   
3. Implémenter une fonction qui prend en argument un entier $m$, un module de chiffrement $n$, et un exposant $e$, et qui retourne l'exponentiation de $m$ par $e$ modulo $n$.
Attention : Ne pas calculer directement $m^{e}$ car ce nombre peut être très grand. Calculez itérativement $m^{i+1} \equiv m * m^{i}(\bmod n)$, avec $e$ étapes de calcul.

### 2.4 Exemple d'exécution de RSA

Cette section présente une exécution complète du protocole RSA, dans laquelle Bob souhaite envoyer un message à Alice.

Génération des clés : Alice commence par qénérer les clés privée et publique.

1. Alice génère deux nombres premiers $p=11$ et $q=17$.
2. Le module de chiffrement est donc $n=p q=187$.
3. L'indicatrice d'Euler de $n$ est alors $\phi(n)=(p-1)(q-1)=160$.
4. Alice choisit l'exposant de chiffrement $e=7$, avec $e$ inférieur à 160 et premier avec 160 .
5. L'exposant de déchiffrement est $d=23$, c'est-à-dire l'inverse modulaire de 7 modulo 160 .

Les clés générées par Alice sont $(187,7)$ pour le chiffrement et $(187,23)$ pour le déchiffrement. Alice transmet la clé publique $(187,7)$ à Bob.

Chiffrement message : Bob souhaite envoyer le message $m=30$. Bob calcule le message chiffré $c \equiv 30^{7} \equiv 123(\bmod 187)$. Puis Bob transmet ce message chiffré $c=123$ à Alice.

Déhiffrement message : Alice reçoit le message chiffré $c=123$. Alice déchiffre le message en calculant $m \equiv 123^{23} \equiv 30(\bmod 187)$. Alice a retrouvé le message original.

### 2.5 Justification de Validité

Dans cette section, nous démontrons que la génération des clés et les fonctions de chiffrement du protocole permettent bien de chiffrer et déchiffrer.

Considérons les clés $(n, e)$ et $(n, d)$, et un message $m$ tel que

$$
m^{e} \equiv c \quad(\bmod n)
$$

Le message $c$ est donc le message chiffré selon le protocole RSA.
Pour être valide, ce protocole doit garantir le déchiffrement :

$$
\begin{align*}
c^{d} & \equiv m \quad(\bmod n)  \tag{1}\\
\text { c'est-à-dire }, \quad m^{e d} & \equiv m \quad(\bmod n)
\end{align*}
$$

Montrons donc que l'Equation 1 est vérifiée à l'aide de 4 arguments.
Premier argument : petit théorème de Fermat avec $p, q$, et $m$.
L'entier $p$ est premier et $m$ n'est pas un multiple de $p$. D'après le petit théorème de Fermat, nous obtenons

$$
\begin{equation*}
m^{p-1} \equiv 1 \quad(\bmod p) \tag{2}
\end{equation*}
$$

De même avec $q$, nous obtenons

$$
\begin{equation*}
m^{q-1} \equiv 1 \quad(\bmod q) \tag{3}
\end{equation*}
$$

Deuxième argument : lien entre les exposants et $d$.
Après avoir choisi l'exposant de chiffrement $e$, le protocole RSA calcule l'inverse modulaire modulo $\phi(n)=(p-1)(q-1)$. Par définition, on a donc

$$
e d \equiv 1 \quad(\bmod (p-1)(q-1))
$$

Ceci implique qu'il existe un entier relatif $k$ tel que

$$
\begin{equation*}
e d=1+k(p-1)(q-1) \tag{4}
\end{equation*}
$$

Troisième argument : Calcul de $m^{e d}$.
D'après l'Equation 4, on a :

$$
\begin{aligned}
m^{e d} & =m^{1+k(p-1)(q-1)} \\
& =m\left(m^{(p-1)}\right)^{k(q-1)}
\end{aligned}
$$

En passant à la notation modulo $p$, on obtient

$$
m^{e d} \equiv m\left(m^{(p-1)}\right)^{k(q-1)} \quad(\bmod p)
$$

Puis, avec l'Equation 2, on obtient

$$
\begin{align*}
m^{e d} & \equiv m(1)^{k(q-1)} \quad(\bmod p) \\
& =m \quad(\bmod p) \tag{5}
\end{align*}
$$

De même pour $q$, en passant à la notation modulo $q$ et avec l'Equation 3, on obtient

$$
\begin{align*}
m^{e d} & \equiv m *(1)^{k(p-1)} \quad(\bmod q) \\
& =m \quad(\bmod q) \tag{6}
\end{align*}
$$

Les Equation 5 et 6 nous permettent de dire que $\left(m^{e d}-m\right)$ est multiple de $p$ et multiple de $q$.
Quatrième argument: Gauss et $\left(m^{e d}-m\right)$.
On a montré que $\left(m^{e d}-m\right)$ est multiple de $p$ et multiple de $q$.
Or, $p$ et $q$ sont des nombres premiers, donc d'après la propriété associée au théorème de Gauss, $\left(m^{e d}-m\right)$ est un multiple de $p * q$, c'est-à-dire multiple de $n$.
Ceci implique

$$
\begin{align*}
m^{e d}-m & \equiv 0 \quad(\bmod n) \\
m^{e d} & \equiv m \quad(\bmod n) \tag{7}
\end{align*}
$$

Le protocole RSA est donc valide, il garantit le déchiffrement d'un message chiffré.

### 2.6 Implémentation

1. Ecrire une fonction qui génère aléatoirement un nombre premiers, compris entre deux bornes données en argument.

- Aide en C : la fonction $\operatorname{rand}()$ retourne un entier alétoire entre 0 et une constante "RAND_MAX". Pour obtenir un entier $p$ inférieur à borne_sup, on peut calculer : $p=\operatorname{rand}() \%$ borne_sup.
- Aide en Python : la fonction $\operatorname{randint}(a, b)$ de la bibliothèque Random génère un entier entre $a$ et $b$.

2. Ecrire une fonction qui prend un entier $\phi(n)$ en argument, et génère aléatoirement un entier inférieur $\phi(n)$ et premier avec $\phi(n)$.
3. Ecrire une fonction qui calcul l'inverse modulaire d'un entier a modulo $n$ (voir support de cours n ${ }^{\circ} 4$ ).
4. Implémenter une fonction qui prends en argument un message sous forme d'un entier $m$, un module de chiffrement $n$, et un exposant $e$, et qui retourne l'exponentiation de $m$ par l'exposant $e$ modulo $n$.
5. Ecrire une fonction qui génère les clés publiques et privées, puis chiffre un message et le déchiffre à l'aide des clés générées.

## Source :

Éric Barbazo, Christophe Barnet, Martial Baheux, Dominique Grihon, Barbazo
Mathématiques Expertes Terminale, édition 2020, Hachette Éducation.

## Exercices supplémentaires :

https://www.bibmath.net/ressources/index.php?action=affiche\&quoi=bde/arithm/ congruence\&type=fexo

#### Un exemple **concret** et **chiffré pas à pas** à partir du message :

Exemple complet du chiffrement RSA, mais cette fois avec le message :

> **"Delta Charlie a confirmé l’approche."**

Nous allons l’illustrer avec un mini-RSA pour la clarté (valeurs réduites à des tailles pédagogiques).

Message d'origine :
Delta Charlie a confirmé l’approche.
➡ On simplifie à : DELTA CHARLIE

---

Step 1: Chosen primes p=17, q=11
Step 2: Calculate n = p * q = 187
Step 3: Calculate φ(n) = (p-1)*(q-1) = 160
Step 4: Chosen e = 7
Step 5: Calculate d = e^(-1) mod φ(n) = 23
Public Key: (187, 7)
Private Key: (187, 23)


### 📤 Étape 2 : Chiffrement (par Bob)

Le message est :
**"Delta Charlie a confirmé l’approche."**

#### 🔢 Encodage simple (ASCII caractère par caractère)

On prend la version simplifiée :
on encode chaque lettre en majuscule sans accent et sans ponctuation (par exemple, via position dans l’alphabet ou code ASCII tronqué).

Exemple : D → 4, E → 5, L → 12, T → 20, A → 1
(on peut faire : A=1, B=2, ..., Z=26, espace=0)

> **"DELTA CHARLIE"**
> → devient :
> \[4, 5, 12, 20, 1, 0, 3, 8, 1, 18, 12, 9, 5]

Puis chaque entier $m_i$ est chiffré comme :

$$
c_i = m_i^e \mod n
$$

Par exemple :

* $c_1 = 4^7 \mod 187 = 16384 \mod 187 = 103$
* $c_2 = 5^7 \mod 187 = 78125 \mod 187 = 103$
* etc.

➡️ Le message chiffré est la **suite des $c_i$**.

---

### 📥 Étape 3 : Déchiffrement (par Alice)

Alice utilise sa **clé privée $d = 23$** pour retrouver chaque $m_i$ :

$$
m_i = c_i^d \mod n
$$

Par exemple :

* Si $c_1 = 103$, alors $m_1 = 103^{23} \mod 187 = 4$ (on retrouve D)

---

## ✅ Conclusion

* **Bob** a utilisé la **clé publique d’Alice** pour chiffrer : "Delta Charlie a confirmé l’approche."
* **Alice**, avec sa **clé privée**, peut seule déchiffrer et reconstituer le message.

## Définitions :

Une **clé privée** en cryptographie asymétrique (comme RSA) est un **nombre secret** utilisé pour **déchiffrer** les messages chiffrés à l’aide de la **clé publique** correspondante.

### Concrètement :

* Elle est **connue uniquement du destinataire** (par exemple, Alice).
* Elle permet de **retrouver le message original** à partir du message chiffré.
* En RSA, c’est le couple $(n, d)$, où :

  * $n = p \times q$, le produit de deux grands nombres premiers.
  * $d$ est l’**inverse modulaire** de $e$, l’exposant de chiffrement.

---

### Exemple :

Si la clé publique est :

* $n = 187$
* $e = 7$

Et la clé privée correspondante est :

* $d = 23$

Alors :

* Un message chiffré avec $e = 7$ ne peut être déchiffré **qu’avec $d = 23$**.

---

**Résumé** :

> La clé privée est **le secret qui permet de lire les messages codés**, exactement comme une **clé de coffre-fort** que seule la personne autorisée détient.

La **clé publique** en cryptographie asymétrique (comme RSA) représente un **paire d’informations mathématiques** que tout le monde peut connaître et utiliser pour **chiffrer un message destiné à une personne spécifique**.

---

### 🔐 Définition précise

La **clé publique** est un couple de nombres :

$$
\text{Clé publique} = (n, e)
$$

* $n$ : le **module**, produit de deux grands nombres premiers $p$ et $q$
* $e$ : l’**exposant de chiffrement**, choisi de façon à être premier avec $\varphi(n)$

> Elle est appelée **publique** car elle peut être **diffusée librement** sur Internet, dans des annuaires, etc.

---

### 💡 À quoi sert-elle concrètement ?

* Elle permet **à n’importe qui** de **chiffrer** un message destiné au propriétaire de la clé.
* **Seul le détenteur de la clé privée correspondante** peut déchiffrer ce message.

---

### 📦 Exemple simple

Si la clé publique d’Alice est :
$(n = 187, e = 7)$

Alors n’importe qui (comme Bob) peut chiffrer un message $m$ en calculant :

$$
c = m^e \mod n
$$

Mais seul Alice, qui connaît l’exposant privé $d$, pourra retrouver $m$ avec :

$$
m = c^d \mod n
$$

---

### 🔐 En résumé

| Élément          | Utilisateur     | Utilisation         | Peut être public ? |
| ---------------- | --------------- | ------------------- | ------------------ |
| Clé **publique** | Tous            | Pour **chiffrer**   | ✅ Oui              |
| Clé **privée**   | Le destinataire | Pour **déchiffrer** | ❌ Non (secrète)    |

Souhaitez-vous que j’explique aussi comment cette clé est techniquement construite ?
